In [261]:
import pandas as pd
import plotly.express as px
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score
from typing import List, Tuple
from xgboost import XGBRegressor 
import uuid
import datetime 
import time
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit, GridSearchCV, train_test_split
from utils.output_analysis import plot_decision_tree_importance, plot_player_value_trends, save_output_tables
import optuna



In [262]:
start = time.time()

In [251]:
pdf_mvp = pd.read_csv("data/intermediate/time_series_model_data_prep.csv") 

In [252]:
test_start = 2023
version  = "v0.0.4"

features = [
    "value_last_year",
    "age_last_year",
    "pos_Attack",
    "pos_Defender",
    "pos_Goalkeeper",
    "pos_Midfield",
    "contract_years_left",
]

subpos_features = [col for col in pdf_mvp.columns if col.startswith("subpos_")]
features += subpos_features

#test_players = pdf_mvp[pdf_mvp["year"] == test_start]["player_id"].unique()

train_df = pdf_mvp[(pdf_mvp["year"] < test_start)]
test_df = pdf_mvp[(pdf_mvp["year"] == test_start)]




In [253]:
def objective(
    trial: optuna.Trial,
    X: pd.DataFrame,
    y: pd.Series,
    tscv: TimeSeriesSplit,
) -> float:
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),  # Number of trees
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),  # Learning rate
        "max_depth": trial.suggest_int("max_depth", 1, 7),  # Tree depth
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),  # Subsampling fraction
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),  # Min loss reduction for split
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),  # L1 regularization
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10.0),  # L2 regularization
        "random_state": 42,
        "objective": "reg:squarederror",
        "tree_method": "hist",
        "verbosity": 0,
    }

    rmse_scores = []

    for train_idx, val_idx in tscv.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        preds = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, preds)
        rmse_scores.append(rmse)

    return sum(rmse_scores) / len(rmse_scores)  # Average RMSE across folds


def predict_future_values(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    prediction_years: List[int],
    features: List[str],
    n_trials: int = 500,
) -> Tuple[pd.DataFrame, XGBRegressor, pd.Series]:
    """
    Predicts future player values for specified years using XGBoost and Optuna hyperparameter tuning.

    Args:
        train_df: Training DataFrame with features and target.
        test_df: Initial test DataFrame to predict from (starting year).
        prediction_years: List of years to predict, starting with the base year.
        features: List of feature column names to use.
        n_trials: Number of Optuna trials for hyperparameter search.

    Returns:
        Tuple of:
            - DataFrame with predicted values for all prediction years concatenated.
            - Trained XGBRegressor using best Optuna params.
            - y_test from the base year (only valid for the first year).
    """
    target = "market_value_in_million_eur"

    X_train = train_df[features]
    y_train = train_df[target]

    tscv = TimeSeriesSplit(n_splits=3)

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, X_train, y_train, tscv), n_trials=n_trials, show_progress_bar=False)

    print(f"Best trial params: {study.best_trial.params}")
    print(f"Best trial RMSE: {study.best_value:.4f}")

    # Train final model on all training data with best params
    best_params = study.best_trial.params
    # Add fixed params required by XGBRegressor
    best_params.update({
        "random_state": 42,
        "objective": "reg:squarederror",
        "tree_method": "hist",
        "verbosity": 0,
    })

    model = XGBRegressor(**best_params)
    model.fit(X_train, y_train)

    predictions = []
    current_df = test_df.copy()

    for i, year in enumerate(prediction_years):
        X_test = current_df[features]
        current_df["predicted_value"] = model.predict(X_test)
        current_df["year"] = year

        if i == 0:
            y_test = current_df[target]
            rmse = root_mean_squared_error(y_test, current_df["predicted_value"])
            r2 = r2_score(y_test, current_df["predicted_value"])
            print(f"{year} RMSE: {rmse:.2f}")
            print(f"{year} R²: {r2:.3f}")

            fig = px.scatter(current_df, x="predicted_value", y=target, hover_data=["name", "age"])
            fig.show()
        else:
            current_df["age"] = current_df["age_last_year"] + 1

        # Reduce contract years left by 1, never below 0
        if "contract_years_left" in current_df.columns:
            current_df["contract_years_left"] = (current_df["contract_years_left"] - 1).clip(lower=0)

        current_df["age_last_year"] = current_df["age"]
        current_df["value_last_year"] = current_df["predicted_value"]

        predictions.append(current_df[["player_id", "year", "age", "predicted_value"]].copy())

        if i < len(prediction_years) - 1:
            pos_cols = [col for col in current_df.columns if col.startswith("pos_")]
            subpos_cols = [col for col in current_df.columns if col.startswith("subpos_")]
            static_cols = pos_cols + subpos_cols

            carry_cols = ["player_id", "value_last_year", "age", *static_cols]
            if "contract_years_left" in current_df.columns:
                carry_cols.append("contract_years_left")
            current_df = current_df[carry_cols].copy()
            current_df.rename(columns={"age": "age_last_year"}, inplace=True)

    all_predictions_df = pd.concat(predictions, ignore_index=True)
    return all_predictions_df, model, y_test


In [254]:
forecast_df, regressor, y_test = predict_future_values(train_df, test_df, [2023, 2024, 2025, 2026, 2027, 2028], features)

[I 2025-07-24 18:29:41,185] A new study created in memory with name: no-name-03e62de4-ec46-4eba-a98f-9cc010b118ce
[I 2025-07-24 18:29:41,469] Trial 0 finished with value: 4.7927880980568895 and parameters: {'n_estimators': 68, 'learning_rate': 0.12654978434433, 'max_depth': 2, 'subsample': 0.8725304104250828, 'colsample_bytree': 0.8088883508576226, 'gamma': 1.517432896958123, 'reg_alpha': 0.43708474335503555, 'reg_lambda': 8.677025612979213}. Best is trial 0 with value: 4.7927880980568895.
[I 2025-07-24 18:29:41,802] Trial 1 finished with value: 5.004718343900908 and parameters: {'n_estimators': 134, 'learning_rate': 0.13500230844812539, 'max_depth': 3, 'subsample': 0.9571664592487529, 'colsample_bytree': 0.8216831888982907, 'gamma': 3.7255475130157873, 'reg_alpha': 0.9894726123041405, 'reg_lambda': 6.596520998078626}. Best is trial 0 with value: 4.7927880980568895.
[I 2025-07-24 18:29:42,214] Trial 2 finished with value: 4.898509109016184 and parameters: {'n_estimators': 155, 'learnin

Best trial params: {'n_estimators': 194, 'learning_rate': 0.20509288648627058, 'max_depth': 2, 'subsample': 0.9898026468847371, 'colsample_bytree': 0.9860848387520088, 'gamma': 0.9323453671730814, 'reg_alpha': 0.747528065482912, 'reg_lambda': 6.6747529133124175}
Best trial RMSE: 4.6046
2023 RMSE: 6.62
2023 R²: 0.826


In [255]:
# Merge with actual market values for evaluation

merged = forecast_df.merge(test_df[["player_id", "name"]], on=["player_id"], how="left")
end = time.time()
diff = end - start

In [256]:
# plot_decision_tree_importance(regressor.best_estimator_, features)

In [257]:
players = [418560, 357662, 565822, 859951, 148455, 274839, 542580, 568177]
plot_player_value_trends(train_df=train_df, merged_df=merged, player_ids=players)


In [258]:
output_df = merged.query("year == 2023")
output_df["actual_value"] = y_test.values


C:\Users\tomkn\AppData\Local\Temp\ipykernel_2016\2121820575.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [259]:
output_df["model_output_id"] = uuid.uuid4()
output_df["model_run_date"] = datetime.datetime.now()
output_df["time_taken_seconds"] = diff
output_df["features_used"] = str(features)
output_df["model_type"] = "XG_Boost"
output_df["split_year"] = test_start
output_df["version"] = version

C:\Users\tomkn\AppData\Local\Temp\ipykernel_2016\1487859311.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tomkn\AppData\Local\Temp\ipykernel_2016\1487859311.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tomkn\AppData\Local\Temp\ipykernel_2016\1487859311.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [260]:
save_output_tables(output_df)